# 📦 Bin Packing problem (BPP)

## Definizione

Il Bin Packing problem può essere definito come segue

> Dati $N$ oggetti da inscatolare ed il "peso" di ciascuno di essi, trovare il numero minimo di scatole necessarie a contenere tutti gli oggetti, nota la capacità massima di una scatola (uguale per tutte).

<img src="../bpp-instance.png" style="margin-left:auto;margin-right:auto;width:50%">

### Domanda

Ti viene in mente un caso in cui il problema così formulato risulterebbe automaticamente _unfeasible_?

`per esempio se esiste un oggetto con peso individuale maggiore della capacità di una scatola`

Facciamo un passo indietro e pensiamo a formulazioni più semplici

### Esercizio (BPP semplificato)
Nel caso in cui:

- la capacità di una scatola è pari a $C$
- il "peso" è uguale per tutti gli oggetti, diciamo $w \leq C$

qual è il numero minimo di scatole necessario a contenere tutti gli oggetti?


`numero minimo di scatole = `$\large \left\lceil \frac{N\cdot w}{C}\right\rceil$

## Formulazione

Di seguito è riportata la formulazione matematica del BPP (in particolare, la _[formulazione debole](https://scipbook.readthedocs.io/en/latest/flp.html#cflp-quality)_).

$\large \min \sum\limits_{j=1}^Ny_j$

s.t.

$\large (1) \sum\limits_{j=1}^Nx_{ij} = 1 \;\;\;\forall i=1,\dots,N$
    
$\large (2) \sum\limits_{i=1}^Nw_ix_{ij} \leq Cy_j \;\;\;\forall j=1,\dots,N$

$\large (3)\;\;x_{ij}, y_j \in \{0, 1\} \;\;\;\forall i, j=1,\dots,N$

### Domanda

1. Chi sono le _variabili decisionali_? 
2. Cosa rappresentano?
3. Cosa esprime il vincolo (3)?

1. `sono` $\large x$ `e` $\large y$

2. `rappresentano`
- $\large y_j=1$ `se la j-esima scatola è utilizzata, `$\large y_j=0$ `altrimenti`
- $\large x_{ij}=1$ `se l'i-esimo oggetto è assegnato alla j-esima scatola, `$\large x_{ij}=0$ `altrimenti`

3. `esprime la scelta di variabili decisionali booleane, che possono quindi assumere solo i valori 0 e 1`

### Domanda

A che famiglia di problemi appartiene il BPP così formulato?

`se `$\large w_i, C\in\mathbb{R}$ ` è un problema MILP, se `$\large w_i, C\in\mathbb{N}$` è un problema ILP`

### Domanda

A tuo parere, perchè _anche_ l'indice $j$ assume valori tra 1 ed $N$?

`perchè a priori N è un upper bound sul numero di scatole necessarie, raggiunto nel caso in cui si inscatola ogni oggetto in una scatola dedicata`

### Domanda

Che cosa esprime la funzione obiettivo?

`esprime l'obiettivo di minimizzare il numero di scatole utilizzate, rappresentato dalla somma delle variabili booleane indicatrici dell'utilizzo di ciascuna scatola`

### Domanda

Che cosa esprime il vincolo (1)?

`esprime la richiesta "logica" che ciascun oggetto sia assegnato ad una ed una sola scatola: fissato l'indice i, deve esistere un unico indice j per cui `$\large x_{ij}=1$

### Domanda 
Che cosa esprime il vincolo (2)?

`esprime la richiesta che il peso totale degli oggetti assegnati ad una data scatola non superi la capacità massima della scatola. 
questo vincolo risulta violato "automaticamente" se esiste un oggetto i* per cui `$\large w_{i^*}>C$

### Esercizio (BONUS)
Riformula il problema con notazione matriciale

$\large \min \left\Vert \boldsymbol{y}\right\Vert_1$

`s.t.`

$\large (1) \;\;\boldsymbol{X}\cdot\mathbb{1} = \mathbb{1}$ dove $\large \mathbb{1}^T = (1,\dots,1)$
    
$\large (2) \;\;\left(C\boldsymbol{y}^T - \boldsymbol{w}^T\boldsymbol{X}\right)\cdot\mathbb{1} = \left\Vert C\boldsymbol{y}^T - \boldsymbol{w}^T\boldsymbol{X}\right\Vert_1$ dove $\large \boldsymbol{w}^T = (w_1,\dots,w_N)$

$\large (3)\;\;\boldsymbol{X}\in\mathcal{M}_{N\times N}(\{0,1\}), \boldsymbol{y} \in \{0, 1\}^N$

## Implementazione

Fonti: 

- https://scipbook.readthedocs.io/en/latest/bpp.html#fig-bpp-sol
- https://developers.google.com/optimization/bin/bin_packing

da ortools importiamo il modulo `pywraplp` che sta per "python wrapper (for) linear programming"

In [ ]:
from ortools.linear_solver import pywraplp

### Dati

Ricreiamo i dati del problema della figura sopra

In [ ]:
def create_data_model():
    data = {}
    weights = [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]
    data['oggetti'] = list(range(1, len(weights) + 1))
    data['pesi'] = weights    
    data['scatole'] = data['oggetti']
    data['capacità_scatole'] = 9
    return data

In [ ]:
data = create_data_model()

In [ ]:
for k, v in data.items():
    print(f"{k.rjust(16)}: {v}")

### Inizializzazione del problema

inizializziamo un solver, scegliendo [SCIP](https://www.scipopt.org/) (non è l'unica possibilità, vedi [approfondimento](https://developers.google.com/optimization/mip/mip))

In [ ]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### Variabili decisionali
Definiamo le variabili decisionali del BPP:
- per comodità le salviamo in un dizionario
- per aggiungere una variabile usiamo la classe `BoolVar` del solver

In [ ]:
x = {}
for i in data['oggetti']:
    for j in data['scatole']:
        x[(i, j)] = solver.BoolVar(f'x_{i}_{j}')

y = {}
for j in data['scatole']:
    y[j] = solver.BoolVar(f'y_{j}')

Guardiamo come è fatta $\large x$

In [ ]:
# x

Controlliamo di che tipo sono i valori del dizionario

In [ ]:
type(x[(1,1)])

### Vincoli
Aggiungiamo (letteralmente! tramite il metodo `Add`) i vincoli al solver

In [ ]:
# vincolo (1)
for i in data['oggetti']:
    solver.Add(sum(x[i, j] for j in data['scatole']) == 1)

# vincolo (2)
for j in data['scatole']:
    solver.Add(
        sum(x[(i, j)] * data['pesi'][i-1] for i in data['oggetti']) <= y[j] * data['capacità_scatole'])

### Funzione obiettivo
Dichiariamo la funzione obiettivo e il _senso_ dell'ottimizzazione (min/max)

In [ ]:
solver.Minimize(solver.Sum([y[j] for j in data['scatole']]))

### Risoluzione

In [ ]:
import time

tic = time.time()
status = solver.Solve()
toc = time.time()
print(f"Stato: {status}")

Il solver sembra aver risolto il problema molto in fretta. Sinceriamoci del significato dello _stato_ del problema a valle della risoluzione

In [ ]:
nomi_stati = {
    # optimal.
    pywraplp.Solver.OPTIMAL: 'OPTIMAL',
    # feasible, or stopped by limit.
    pywraplp.Solver.FEASIBLE: 'FEASIBLE',
    # proven infeasible.
    pywraplp.Solver.INFEASIBLE: 'INFEASIBLE',
    # proven unbounded.
    pywraplp.Solver.UNBOUNDED: 'UNBOUNDED',
    # abnormal, i.e., error of some kind.
    pywraplp.Solver.ABNORMAL: 'ABNORMAL',
    # the model is trivially invalid (NaN coefficients, etc).
    5: 'MODEL_INVALID',
    # not been solved yet.
    pywraplp.Solver.NOT_SOLVED: 'NOT_SOLVED'
}

In [ ]:
print(f"Stato: {nomi_stati[status]}")

### Valutazione e interpretazione

Dobbiamo ora capire che valori hanno assunto le variabili decisionali a valle della risoluzione.

Utilizziamo il metodo `solution_value()` disponibile per ciascuna variabile.

In [ ]:
scatole_utilizzate = {}

for j in data['scatole']:
    if y[j].solution_value() == 1:
        scatole_utilizzate[j] = {'oggetti': [], 'peso_totale': 0}
        for i in data['oggetti']:
            if x[i, j].solution_value() == 1:
                scatole_utilizzate[j]['oggetti'].append(i)
                scatole_utilizzate[j]['peso_totale'] += data['pesi'][i-1]
        scatole_utilizzate[j]['saturazione'] = f"{scatole_utilizzate[j]['peso_totale']/data['capacità_scatole']*100:.0f}%"
        
display(scatole_utilizzate)
print()
print(f'Numero di scatole utilizzate: {len(scatole_utilizzate)}')
print(f'Tempo computazionale [s]: {toc - tic}')

In [ ]:
from IPython.display import Image
Image('../bpp-solution.png')

### Domanda

Secondo te la soluzione trovata è unica?

`no, è possibile anche spostare uno degli oggetti di peso 2 nella scatola con il solo oggetto di peso 6`

# BONUS: use case reale `MiMocko` 🛵

Le batterie degli scooter elettrici MiMocko vengono sostituite ogni notte! 🔋

- alcuni tecnici compiono ricognizioni notturne per sostituire le batterie di tutti i veicoli della flotta.

- per una scelta di ecosostenibilità, il trasporto delle batterie avviene tramite casse di legno che possono sostenere un peso massimo complessivo di 120 kg (altrimenti si rompe...)

MiMocko ha via via aggiornato il suo parco veicoli, e gli scooter più recenti hanno una batteria più leggera rispetto a quelli più datati.

Le caratteristiche delle batterie della flotta MiMocko sono le seguenti:

- batterie da 11 kg per scooter di versione 1
- batterie da 8 kg per scooter di versione 2
- batterie da 6 kg per scooter di versione 3

## Esercizio

Risolvere tramite l'implementazione di un BPP il problema di determinare il minor numero di casse necessarie al trasporto di _tutte_ le batterie della flotta MiMocko ogni notte.

**Hint**: 

1. per determinare il numero di batterie delle varie versioni, puoi utilizzare il dataset `scooter.csv`
2. sfrutta la traccia del BPP risolto in precedenza per determinare
    - variabili decisionali
    - funzione obiettivo
    - vincoli
    - eventuali parametri

In [ ]:
import pandas as pd
import time
from ortools.linear_solver import pywraplp

In [ ]:
path = '../../../../data'

In [ ]:
scooter = pd.read_csv(f"{path}/scooter.csv", sep=';')

In [ ]:
versioni2quantita = scooter.groupby('Versione n°').size().to_dict()
versioni2quantita

In [ ]:
versioni2pesi = {1: 13, 2: 7, 3: 6}
versioni2pesi

In [ ]:
def create_data_model():
    data = {}
    weights = [peso for versione, peso in versioni2pesi.items() for _ in range(versioni2quantita[versione])]
    data['batterie'] = list(range(1, len(weights) + 1))
    data['pesi'] = weights    
    data['casse'] = data['batterie']
    data['capacità_casse'] = 80
    return data

In [ ]:
data = create_data_model()

In [ ]:
for k, v in data.items():
    print(f"{k.rjust(16)}: {v}")

In [ ]:
solver = pywraplp.Solver.CreateSolver('SCIP')

x = {}
for i in data['batterie']:
    for j in data['casse']:
        x[(i, j)] = solver.BoolVar(f'x_{i}_{j}')

y = {}
for j in data['casse']:
    y[j] = solver.BoolVar(f'y_{j}')
    
# vincolo (1)
for i in data['batterie']:
    solver.Add(sum(x[i, j] for j in data['casse']) == 1)

# vincolo (2)
for j in data['casse']:
    solver.Add(
        sum(x[(i, j)] * data['pesi'][i-1] for i in data['batterie']) <= y[j] * data['capacità_casse'])
    
solver.Minimize(solver.Sum([y[j] for j in data['casse']]))

tic = time.time()
status = solver.Solve()
toc = time.time()

nomi_stati = {
    # optimal.
    pywraplp.Solver.OPTIMAL: 'OPTIMAL',
    # feasible, or stopped by limit.
    pywraplp.Solver.FEASIBLE: 'FEASIBLE',
    # proven infeasible.
    pywraplp.Solver.INFEASIBLE: 'INFEASIBLE',
    # proven unbounded.
    pywraplp.Solver.UNBOUNDED: 'UNBOUNDED',
    # abnormal, i.e., error of some kind.
    pywraplp.Solver.ABNORMAL: 'ABNORMAL',
    # the model is trivially invalid (NaN coefficients, etc).
    5: 'MODEL_INVALID',
    # not been solved yet.
    pywraplp.Solver.NOT_SOLVED: 'NOT_SOLVED'
}

print(f"Stato: {nomi_stati[status]}")

casse_utilizzate = {}

for j in data['casse']:
    if y[j].solution_value() == 1:
        casse_utilizzate[j] = {'batterie': [], 'peso_totale': 0}
        for i in data['batterie']:
            if x[i, j].solution_value() == 1:
                casse_utilizzate[j]['batterie'].append(i)
                casse_utilizzate[j]['peso_totale'] += data['pesi'][i-1]
        casse_utilizzate[j]['saturazione'] = f"{casse_utilizzate[j]['peso_totale']/data['capacità_casse']*100:.0f}%"
        
display(casse_utilizzate)
print()
print(f'Numero di casse utilizzate: {len(casse_utilizzate)}')
print(f'Tempo computazionale [s]: {toc - tic}')

# Approfondimenti

- [BPP con items fragmentation](https://hal.archives-ouvertes.fr/hal-00780434/document)
- [The Optimization process](https://coin-or.github.io/pulp/main/the_optimisation_process.html)
- [MOSEK optimization modeling cookbook](https://docs.mosek.com/modeling-cookbook/index.html)